In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import probit
from statsmodels.formula.api import logit
from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.utils import resample # for bootstrap
import scipy.stats # for confidence interval
!pip install mord # for ordered logit

  Created wheel for mord: filename=mord-0.6-cp36-none-any.whl size=6008 sha256=bf0a1e6749756cee6a26959e336d912323202a09448c1aeed41a4381379296f0
  Stored in directory: /root/.cache/pip/wheels/98/14/b2/244c2cec93a0c6edb29b488bd6b2710ded7e9d457033b86366
Successfully built mord


## Load Data

In [2]:
data = pd.read_csv('https://drive.google.com/uc?export=download&id=1zRUfEsLc0bm3njKL8vzX1mXGjpkAPEHl')

data.head()

,seq,s3,s4,s5,s6,s7,s9,s11,s13,s14,s15,s16,s17,s18,s19a,s19b,s19c,s19d,s20,s23a,s24a,s25a,s26a,s27a,s30a,s30c,s31a,s31c,s32,s33,s34,s35,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,netw,uria,rtdum,bd,mi,old,vr,school,chval,dnotown,dprop
0,2,1,1,1,88,1,1120,0,5,5.0,2,1.0,53.0,8.0,NaN,NaN,NaN,NaN,1.0,M,0,19.0,19.0,0,1938.0,2000.0,2241.0,2000.0,939.0,110.0,0.0,30.0,0.0,0,34.0,2,5,0,22.1,22.1,3,360.0,NaN,110.0,2,0,0,N,0,1,999999.4,92.0,3.9,0,1,1,1,0,15.0,206.7355,0,0
1,3,1,1,1,118,1,1120,0,5,5.0,1,2.0,51.0,8.0,NaN,NaN,NaN,NaN,1.0,U,0,3.0,1.0,0,2638.0,1642.0,2638.0,1642.0,1161.0,128.0,0.0,19.0,0.0,1,46.0,2,2,0,26.5,26.5,2,360.0,NaN,128.0,2,1,0,N,0,0,999999.4,3.0,3.2,0,1,1,0,1,18.0,238.4943,0,0
2,7,1,1,1,185,1,1120,0,5,5.0,1,2.0,99.0,8.0,NaN,NaN,NaN,NaN,1.0,M,0,13.0,14.0,0,5000.0,2500.0,5600.0,2500.0,2011.0,196.0,0.0,24.0,0.0,1,24.0,2,1,0,24.8,37.2,2,360.0,NaN,201.0,2,0,0,N,0,0,999999.4,22.0,3.2,0,1,1,1,0,12.0,200.1977,0,0
3,9,1,1,1,185,1,1120,0,5,5.0,1,2.0,110.0,1.0,NaN,NaN,NaN,NaN,1.0,M,2,3.0,3.0,0,7667.0,0.0,7667.0,0.0,1937.0,210.0,0.0,11.0,1.0,1,20.0,2,1,0,25.0,32.0,2,360.0,NaN,215.0,2,1,0,N,0,0,1.0,241.0,4.3,0,0,1,0,1,12.0,193.8895,0,0
4,10,1,1,1,330,1,1120,0,5,5.0,1,2.0,129.0,0.0,NaN,NaN,NaN,NaN,1.0,M,1,17.0,11.0,0,6083.0,4250.0,6083.0,4250.0,3608.0,550.0,0.0,222.0,1.0,1,6.0,1,1,0,35.0,36.0,2,360.0,NaN,550.0,2,0,0,N,0,0,2.0,365.0,3.2,0,0,1,1,0,20.0,257.0874,0,0


In [0]:
# restrict to male applicants

data = data.loc[data['s15']==1]

In [0]:
# drop self-employed people

data = data.loc[data['s27a']==0]

In [0]:
# drop if private mortgage insurance denied
data = data.loc[data['s53']==0]

In [7]:
# no public recorded default at time of application
data = data.loc[data['s44']==0]

data.describe()

,seq,s3,s4,s5,s6,s7,s9,s11,s13,s14,s15,s16,s17,s18,s19b,s19c,s19d,s20,s24a,s25a,s26a,s27a,s30a,s30c,s31a,s31c,s32,s33,s34,s35,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s50,s51,s52,s53,s55,s56,s57,netw,uria,rtdum,bd,mi,old,vr,school,chval,dnotown,dprop
count,1484.000000,1484.0,1484.0,1484.000000,1484.000000,1484.000000,1484.0,1484.000000,1484.000000,1483.000000,1484.0,1483.000000,1484.000000,1431.000000,100.000000,27.000000,5.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.0,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.0,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.0,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.000000,1484.0
mean,2311.724394,1.0,1.0,1.033693,142.648922,1.182615,1120.0,0.135445,4.800539,5.396494,1.0,2.339852,12879.106873,1.940601,4.470000,4.333333,5.000000,1348.716173,0.803908,15509.118888,20221.836569,0.0,4239.906779,1484.722682,4795.990768,1607.261739,1490.278915,2890.572642,5.435759,4806.980404,2696.905391,2.292453,687.447776,1.696765,1.974394,0.0,25.118633,32.592257,1.701482,1688.542318,204.756873,1.783693,0.229111,0.0,0.035040,0.037062,112535.320620,221.353135,3.755458,0.044474,0.440027,0.863208,0.419811,0.400270,6080.078908,7668.635765,0.030997,0.0
std,1307.520359,0.0,0.0,0.194869,83.690460,0.551777,0.0,0.342313,0.599483,1.352476,0.0,0.795871,112453.894665,2.956556,3.059857,2.868932,2.828427,36698.716431,1.122905,123565.325909,140783.672332,0.0,4236.911276,1786.925228,4821.385309,2142.421665,845.008485,51854.854198,107.146686,68535.607824,51864.765469,29.882587,25958.339539,0.536933,1.555238,0.0,6.961485,7.959923,0.459233,36686.281067,163.541408,0.411865,0.420402,0.0,0.183944,0.188977,316127.667392,1021.902150,1.988735,0.206216,0.496558,0.343744,0.497775,0.490118,77664.536167,85782.381828,0.173369,0.0
min,3.000000,1.0,1.0,1.000000,2.000000,1.000000,1120.0,0.000000,3.000000,1.000000,1.0,1.000000,5.000000,0.000000,1.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.0,0.000000,0.000000,1.000000,6.000000,25.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,-7919.000000,1.800000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,-8.333333,0.000000,0.0
25%,1160.000000,1.0,1.0,1.000000,100.000000,1.000000,1120.0,0.000000,5.000000,5.000000,1.0,2.000000,48.000000,0.000000,1.000000,1.500000,3.000000,1.000000,0.000000,5.000000,2.000000,0.0,2473.000000,0.000000,2708.750000,0.000000,1032.500000,127.975000,0.000000,19.000000,1.000000,1.000000,7.000000,1.000000,1.000000,0.0,21.400000,28.000000,1.000000,360.000000,133.000000,2.000000,0.000000,0.0,0.000000,0.000000,1.000000,40.000000,3.100000,0.000000,0.000000,1.000000,0.000000,0.000000,13.000000,215.406800,0.000000,0.0
50%,2439.500000,1.0,1.0,1.000000,125.000000,1.000000,1120.0,0.000000,5.000000,5.000000,1.0,2.000000,63.000000,1.000000,4.000000,4.000000,3.000000,1.000000,0.000000,9.000000,4.000000,0.0,3265.500000,1251.500000,3640.000000,1346.500000,1306.000000,160.000000,0.000000,37.000000,1.000000,1.000000,12.000000,2.000000,1.000000,0.0,26.000000,33.000000,2.000000,360.000000,165.000000,2.000000,0.000000,0.0,0.000000,0.000000,2.000000,86.126500,3.200000,0.000000,0.000000,1.000000,0.000000,0.000000,16.000000,233.000600,0.000000,0.0
75%,3364.000000,1.0,1.0,1.000000,160.000000,1.000000,1120.0,0.000000,5.000000,5.000000,1.0,2.000000,88.000000,3.000000,7.500000,6.000000,7.000000,1.000000,2.000000,15.000000,9.000000,0.0,4666.000000,2362.250000,5200.000000,2460.500000,1666.000000,226.000000,0.000000,80.625000,2.000000,1.000000,19.000000,2.000000,2.000000,0.0,29.000000,37.000000,2.000000,360.000000,233.000000,2.000000,0.000000,0.0,0.000000,0.000000,3.000000,201.250000

## Raw comparison

In [0]:
# create a dummy variable, black using s13

# practice

idx = data.loc[data['s13']==3].index
data.loc[idx,'black']=1
data['black'].fillna(0, inplace=True)

In [0]:
# create a dummy variable, denial using s7

#practice

idx = data.loc[data['s7']==3].index
data.loc[idx,'denial']=1
data['denial'].fillna(0, inplace=True)

In [11]:
# calculate denial rates for black and non-black applicants

#practice

data.loc[data['black']==0,'denial'].describe()



count    1336.000000
mean        0.063623
std         0.244171
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: denial, dtype: float64

In [12]:
formula = 'denial ~ black'
results = ols(formula, data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 denial   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     36.90
Date:                Thu, 06 Feb 2020   Prob (F-statistic):           1.58e-09
Time:                        19:19:04   Log-Likelihood:                -129.89
No. Observations:                1484   AIC:                             263.8
Df Residuals:                    1482   BIC:                             274.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0636      0.007      8.799      0.0

## Add controls

In [0]:
# loan to value ratio
data['ltv'] = data['s6']/data['s50']

# dummy variable, single
idx = data.loc[data['s23a']=="U"].index

data.loc[idx, 'single'] = 1

data['single'].fillna(0, inplace=True)

# dummy variable, high school
idx = data.loc[data['school'] >= 12].index

data.loc[idx, 'highschool'] = 1

data['highschool'].fillna(0, inplace=True)

# dummy variable, condo
idx = data.loc[data['s51']==1].index

data.loc[idx, 'condo'] = 1

data['condo'].fillna(0, inplace=True)

## OLS

control for

- debt to income ratio (s46) 

- housing expense to income (s45)

- ltv

- credit history (s42 and s43)

- single

- highschool

- unemployment rate (uria)

- condo



In [14]:
#practice

formula = 'denial ~ black + s46 + s45 + ltv + s43 + s42 + single + highschool + uria + condo '
results = ols(formula, data).fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                 denial   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     16.77
Date:                Thu, 06 Feb 2020   Prob (F-statistic):           4.68e-29
Time:                        19:25:32   Log-Likelihood:                -68.150
No. Observations:                1484   AIC:                             158.3
Df Residuals:                    1473   BIC:                             216.6
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0064      0.069     -0.093      0.9

## Logit

In [15]:
results = logit(formula, data).fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.229243
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                 denial   No. Observations:                 1484
Model:                          Logit   Df Residuals:                     1473
Method:                           MLE   Df Model:                           10
Date:                Thu, 06 Feb 2020   Pseudo R-squ.:                  0.1591
Time:                        19:27:58   Log-Likelihood:                -340.20
converged:                       True   LL-Null:                       -404.54
Covariance Type:            nonrobust   LLR p-value:                 8.622e-23
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.0164      0.977     -6.158      0.000      -7.931      -4.102
black          0.8214      0.

In [16]:
# marginal effect
results.get_margeff(at='mean', method='dydx', atexog='black', dummy=True).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 denial
Method:                          dydx
At:                              mean
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
black          0.0533      0.023      2.353      0.019       0.009       0.098
s46            0.0033      0.001      4.606      0.000       0.002       0.005
s45           -0.0005      0.001     -0.593      0.553      -0.002       0.001
ltv            0.0713      0.036      1.987      0.047       0.001       0.142
s43            0.0139      0.003      5.180      0.000       0.009       0.019
s42            0.0195      0.010      2.017      0.044       0.001       0.038
single         0.0336      0.014      2.443      0.015       0.007       0.060
highschool    -0.1756      0.086     -2.035      0.042      -0.345      -0.006
uria           0.0024      0.002      1.061      0.289      -0.002       0.007
condo         -0.0015      0.012     -0.129      0.897      -0.024       0.021
==============================================================================
"""

### predict

In [0]:
exog_var= ['black', 's46', 's45', 'ltv', 's43', 's42', 'single', 'highschool', 'uria','condo']

data['denial_predict'] = results.predict(data[exog_var])

# data.loc[data['black']==0, 'denial_predict'].describe()

In [18]:
data['denial_predict'].describe()

count    1484.000000
mean        0.077493
std         0.095677
min         0.001934
25%         0.026790
50%         0.045841
75%         0.087536
max         0.870515
Name: denial_predict, dtype: float64

## Another way to do logit: sklearn

In [19]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept=True, solver='liblinear', C=1e8)
results = logreg.fit(data[exog_var], data['denial'] )
print(results.coef_)

[[ 0.82308062  0.06865824 -0.01038576  1.48216332  0.29153785  0.40834698
   0.61383589 -1.723281    0.05121266 -0.03227454]]


Results are a little bit different from statsmodels because of regularization.

## Logit with bootstrap standard errors

In [0]:
def logit_wrap(formula, moddata):
    return logit(formula, moddata).fit()

In [0]:
# a function to run bootstraping with given number of iterations for given model

def bootstrap_with_model(func, nrep, formula, moddata):
    # func - input model function
    # nrep - number of iterations
    # formula - given formula for the model, required by statsmodels.formula.api
    # moddata - data used to run the model
    
    mod = func(formula, moddata)
    
    # store parameters of model without bootstrapping
    params0 = mod.params

    params0.name = 'Observed coef.'

    # run bootstrap with nrep number of iterations 
    # store parameters of each iteration as pd.Series in the list, params_list
    # params_list will be used to concatenate all pd.Series in a more efficient way by pd.concat than pd.append
    params_list = []

    for i in range(nrep):
        data_bs = resample(moddata, replace=True)

        temp_n = func(formula, data_bs).params

        params_list.append(temp_n)

        i+=1

    params = pd.concat(params_list, axis=1)

    # find standard erros
    errors = params.sub(params.mean(axis=1), axis=0)

    se = np.sqrt(((errors**2).sum(axis=1))/(nrep-1))

    se.name = 'Bootstrap Std. Err.'

    # find z score
    z = params0/se

    z.name = 'z'
    
    # find p-value
    p_values = scipy.stats.norm.sf(abs(z))*2

    p_values = pd.Series(np.round(p_values, 3), index=z.index, name='P>|z|')
    
    # find 95% confidence interval
    CI = scipy.stats.norm.interval(0.95, loc=params0, scale=se)

    CI_lower = pd.Series(CI[0], index=z.index, name='95% CI,lower')

    CI_high = pd.Series(CI[1], index=z.index, name='95% CI,high')
    
    # return:
    #    mod - the model run without bootstrapping, used for marginal effects later
    #    pd.concat returns the result table
    return mod, pd.concat([params0, se, z, p_values, CI_lower, CI_high], axis=1)

In [0]:
eqn = 'denial ~ black + s46 + s45 + ltv + s43 + s42 + single + highschool + uria + condo'

In [0]:
%%capture 
# %%capture hides tedious text output from iterating model functions in statsmodels.formula.api

logit, res1 = bootstrap_with_model(logit_wrap, 50, eqn, data)

In [24]:
res1

,Observed coef.,Bootstrap Std. Err.,z,P>|z|,"95% CI,lower","95% CI,high"
Intercept,-6.016438,0.956925,-6.287260,0.000,-7.891977,-4.140899
black,0.821419,0.283965,2.892682,0.004,0.264859,1.377980
s46,0.068673,0.015579,4.408136,0.000,0.038139,0.099207
s45,-0.010397,0.016275,-0.638840,0.523,-0.042295,0.021501
ltv,1.494197,0.795701,1.877837,0.060,-0.065348,3.053742
s43,0.291527,0.049670,5.869258,0.000,0.194175,0.388878
s42,0.408018,0.213971,1.906887,0.057,-0.011357,0.827393
single,0.613845,0.214867,2.856853,0.004,0.192712,1.034977
highschool,-1.726979,0.448294,-3.852336,0.000,-2.605619,-0.848339
uria,0.051162,0.056189,0.910526,0.363,-0.058967,0.161290


In [25]:
# marginal effect
logit.get_margeff(at='mean', method='dydx', atexog='black', dummy=True).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 denial
Method:                          dydx
At:                              mean
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
black          0.0533      0.023      2.353      0.019       0.009       0.098
s46            0.0033      0.001      4.606      0.000       0.002       0.005
s45           -0.0005      0.001     -0.593      0.553      -0.002       0.001
ltv            0.0713      0.036      1.987      0.047       0.001       0.142
s43            0.0139      0.003      5.180      0.000       0.009       0.019
s42            0.0195      0.010      2.017      0.044       0.001       0.038
single         0.0336      0.014      2.443      0.015       0.007       0.060
highschool    -0.1756      0.086     -2.035      0.042      -0.345      -0.006
uria           0.0024      0.002      1.061      0.289      -0.002       0.007
condo         -0.0015      0.012     -0.129      0.897      -0.024       0.021
==============================================================================
"""

## Probit with bootstrap standard errors

In [0]:
# practice

def probit_wrap(formula, moddata):
    return probit(formula, moddata).fit()

In [0]:
%%capture 
# use %% capiture for the same reason as above.

probit, res2 = bootstrap_with_model(probit_wrap, 50, eqn, data)

In [28]:
res2

,Observed coef.,Bootstrap Std. Err.,z,P>|z|,"95% CI,lower","95% CI,high"
Intercept,-2.941185,0.457707,-6.425910,0.000,-3.838274,-2.044095
black,0.436081,0.134515,3.241868,0.001,0.172436,0.699726
s46,0.032978,0.008918,3.697849,0.000,0.015499,0.050457
s45,-0.007046,0.009083,-0.775728,0.438,-0.024847,0.010756
ltv,0.685613,0.399230,1.717338,0.086,-0.096864,1.468089
s43,0.154868,0.027359,5.660589,0.000,0.101245,0.208490
s42,0.190049,0.115274,1.648666,0.099,-0.035885,0.415983
single,0.312680,0.125458,2.492316,0.013,0.066788,0.558573
highschool,-0.936817,0.298929,-3.133911,0.002,-1.522707,-0.350927
uria,0.024773,0.028168,0.879470,0.379,-0.030435,0.079981


In [29]:
# marginal effect
probit.get_margeff(at='mean', method='dydx', atexog='black', dummy=True).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                 denial
Method:                          dydx
At:                              mean
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
black          0.0625      0.026      2.405      0.016       0.012       0.113
s46            0.0036      0.001      4.292      0.000       0.002       0.005
s45           -0.0008      0.001     -0.771      0.441      -0.003       0.001
ltv            0.0747      0.039      1.918      0.055      -0.002       0.151
s43            0.0169      0.003      5.253      0.000       0.011       0.023
s42            0.0207      0.011      1.850      0.064      -0.001       0.043
single         0.0383      0.015      2.486      0.013       0.008       0.069
highschool    -0.1932      0.091     -2.122      0.034      -0.372      -0.015
uria           0.0027      0.003      0.991      0.322      -0.003       0.008
condo       3.721e-06      0.014      0.000      1.000      -0.027       0.027
==============================================================================
"""

## Multinomial logit & Poisson

In [31]:
import statsmodels.discrete.discrete_model as discrete

data['s23a'] .value_counts()

M    1054
U     404
S      26
Name: s23a, dtype: int64

In [32]:
exog_var= ['black', 's46', 's45', 'ltv', 's43', 's42', 'highschool', 'uria','condo']
results = discrete.MNLogit(data['s23a'], data[exog_var]).fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.644307
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:                   s23a   No. Observations:                 1484
Model:                        MNLogit   Df Residuals:                     1466
Method:                           MLE   Df Model:                           16
Date:                Thu, 06 Feb 2020   Pseudo R-squ.:                 0.03557
Time:                        20:16:48   Log-Likelihood:                -956.15
converged:                       True   LL-Null:                       -991.42
Covariance Type:            nonrobust   LLR p-value:                 8.059e-09
    s23a=S       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
black         -1.1297      1.044     -1.082      0.279      -3.176       0.917
s46           -0.0020      0.

In [34]:
data['s57'] .value_counts()

1.0         571
2.0         432
3.0         205
999999.4    167
4.0          59
5.0          25
0.0          11
6.0           7
10.0          3
7.0           2
11.0          1
8.0           1
Name: s57, dtype: int64

In [39]:
# Poisson

idx = data.loc[data['s57']== 999999.4].index
data.loc[idx, 's57'] = 0

results = discrete.Poisson(data['s57'], data[exog_var]).fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 1.548609
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                    s57   No. Observations:                 1484
Model:                        Poisson   Df Residuals:                     1475
Method:                           MLE   Df Model:                            8
Date:                Thu, 06 Feb 2020   Pseudo R-squ.:               2.685e-05
Time:                        20:27:57   Log-Likelihood:                -2298.1
converged:                       True   LL-Null:                       -2298.2
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
black          0.0056      0.068      0.082      0.935      -0.128       0.139
s46            0.0020      0.

## Ordered logit

In [40]:
data['s42'].value_counts()

2    956
1    495
3     21
4     12
Name: s42, dtype: int64

In [41]:
# !pip install mord
import mord as m

exog_var= ['black', 's46', 's45', 'ltv', 'single', 'highschool', 'uria','condo']

c = m.LogisticAT()
reg = c.fit(data[exog_var], data['s42'])
reg.coef_

array([ 0.41032511, -0.00816282,  0.01939942,  2.24775026,  0.79464431,
       -0.65835867,  0.06555706,  0.35334296])

In [42]:
#predict

#practice

data['predict']=reg.predict(data[exog_var])
data['predict'].value_counts()

2    1315
1     169
Name: predict, dtype: int64